In [1]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob


In [2]:
WEAK = False

x_len = 5000
t_len = 6000

xs = np.arange(x_len)
dt = 1

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]  

if WEAK:
    X, T = np.meshgrid(xs, np.arange(t_len))
    XT = np.asarray([X, T]).T

    pde_lib = ps.WeakPDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatiotemporal_grid=XT,
        is_uniform=True,
        K=1000,
    )
else:
    
    pde_lib = ps.PDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatial_grid=xs,
        include_bias=True,
        is_uniform=True,
    )
    

In [3]:
def printEnsemble(model, cut_off = 1e-3, median= False):
    if median:
        coefs = np.median(model.coef_list, axis=0)[0,:]
    else:
        coefs = np.mean(model.coef_list, axis=0)[0,:]
    above_cut_off = np.argwhere(np.abs(coefs)>cut_off).flatten()
    features =  model.get_feature_names()
    equation_str= "(x0)' = "
    for count, val in enumerate(above_cut_off):
        if count >0:
            equation_str+=" + "
        equation_str+= "{0:.4f}".format(coefs[val])
        equation_str+= " "+features[val]
        
    print(equation_str)

In [4]:
def SINDyImplementEnsemble(dataset, thresh, alph=0.001, n_models=50, time_steps=200, stlsq_max =100,median=True):
    optimizer = ps.EnsembleOptimizer( opt=ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max),bagging=True,  n_models = n_models,n_subset =time_steps)  
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, differentiation_method= ps.differentiation.SmoothedFiniteDifference())
    model.fit(dataset, t=dt,ensemble=True )
    print("Ensemble STLSQ with Threshold "+str(thresh))
    printEnsemble(model,median=median)

## Unfiltered

### Dataset 1

In [5]:
path = "/data/data2/south-data-ejm/hdd/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/"

init = 11
final= 16

time_subsample = 10
full_dat = np.zeros((5000,(12000//time_subsample)*(final-init)))

k = 0
for i in np.arange(init,final):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    
working_dat = full_dat/np.std(full_dat)


In [6]:
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
thresh = 2.e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.2
(x0)' = 0.6408 x0_1 + 0.3550 x0_111


In [8]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6430 x0_1 + 0.3540 x0_111


In [9]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6361 x0_1 + 0.3502 x0_111


In [10]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6372 x0_1 + 0.3528 x0_111


In [11]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6349 x0_1 + 0.3482 x0_111 + 0.0087 x0x0_1


In [12]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6386 x0_1 + 0.3490 x0_111 + 0.0060 x0x0_1 + 0.0069 x0x0x0_1 + 0.0023 x0x0_11 + -0.0030 x0x0_111 + 0.0061 x0x0x0_111 + 0.0025 x0x0_1111


### Dataset 2

In [13]:
time_subsample = 10
full_dat = np.zeros((5000,6000))

k = 0
for i in np.arange(21,26):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    


In [14]:
working_dat = full_dat/np.std(full_dat)#*10**6
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [15]:
thresh = 2.e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.2
(x0)' = 0.6734 x0_1 + 0.3870 x0_111


In [16]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6758 x0_1 + 0.3895 x0_111


In [17]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6657 x0_1 + 0.3809 x0_111


In [18]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6621 x0_1 + 0.3774 x0_111


In [19]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6714 x0_1 + 0.3821 x0_111 + -0.0087 x0x0_111


In [20]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = -0.0016 x0 + -0.0020 x0x0 + 0.6633 x0_1 + -0.0120 x0_11 + 0.3749 x0_111 + -0.0030 x0_1111 + -0.0042 x0x0_1 + 0.0068 x0x0x0_1 + 0.0038 x0x0x0_11 + -0.0118 x0x0_111 + 0.0064 x0x0x0_111 + 0.0019 x0x0x0_1111


## Drift Removal Load

### Dataset 1

In [21]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [22]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6953 x0_1 + 0.3293 x0_111


In [23]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6900 x0_1 + 0.3242 x0_111


In [24]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.7090 x0_1 + 0.3361 x0_111


In [25]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.7112 x0_1 + 0.3399 x0_111 + -0.0157 x0x0x0_1 + -0.0069 x0x0x0_111


### Dataset 2

In [26]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [27]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6697 x0_1 + 0.3139 x0_111


In [28]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6785 x0_1 + 0.3170 x0_111


In [29]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6828 x0_1 + 0.3196 x0_111


In [30]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6810 x0_1 + 0.3209 x0_111 + -0.0058 x0x0x0_1


### Dataset 3

In [31]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [32]:
thresh = 2.5e-1
SINDyImplementEnsemble(working_dat, thresh)

/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.25) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.25) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.25) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.25) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.25) and eliminated all coefficients
  warnings.warn(
/home/ldoyle/packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.25) and eliminated all coefficients
  warnings.warn(


Ensemble STLSQ with Threshold 0.25
(x0)' = 0.6854 x0_1 + 0.3263 x0_111


In [33]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6814 x0_1 + 0.3217 x0_111


In [34]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6829 x0_1 + 0.3248 x0_111


In [35]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6868 x0_1 + 0.3251 x0_111


In [36]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6934 x0_1 + -0.0039 x0_11 + 0.3298 x0_111 + -0.0013 x0_1111 + -0.0054 x0x0x0_1 + 0.0026 x0x0x0_11 + -0.0043 x0x0x0_111


### Dataset 4

In [37]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


## Multifilt

### Dataset 1

In [38]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [39]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9937 x0_1


In [40]:
thresh = 3.5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.035
(x0)' = 0.9942 x0_1 + 0.0480 x0_111 + 0.0452 x0x0x0_111


In [41]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9942 x0_1 + 0.0481 x0_111 + 0.0450 x0x0x0_111


In [42]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9911 x0_1 + 0.0475 x0_111 + 0.0033 x0x0x0_1 + 0.0490 x0x0x0_111


### Dataset 2

In [43]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [44]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9940 x0_1


In [45]:
thresh = 3.5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.035
(x0)' = 0.9947 x0_1 + 0.0563 x0_111 + 0.0476 x0x0x0_111


In [46]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9949 x0_1 + 0.0558 x0_111 + 0.0475 x0x0x0_111


In [47]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9918 x0_1 + 0.0538 x0_111 + 0.0026 x0x0x0_1 + 0.0514 x0x0x0_111


### Dataset 3

In [48]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [49]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9942 x0_1


In [50]:
thresh = 3.5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.035
(x0)' = 0.9948 x0_1 + 0.0593 x0_111 + 0.0457 x0x0x0_111


In [51]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9949 x0_1 + 0.0580 x0_111 + 0.0455 x0x0x0_111


In [52]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9926 x0_1 + 0.0565 x0_111 + 0.0022 x0x0x0_1 + 0.0508 x0x0x0_111


### Dataset 4

In [53]:
working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [54]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9957 x0_1 + 0.1365 x0_111


In [55]:
thresh = 3.5e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.035
(x0)' = 0.9963 x0_1 + 0.1140 x0_111 + 0.0611 x0x0x0_111


In [56]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9962 x0_1 + 0.1131 x0_111 + 0.0626 x0x0x0_111


In [57]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9940 x0_1 + 0.1088 x0_111 + 0.0021 x0x0x0_1 + 0.0708 x0x0x0_111


In [58]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9940 x0_1 + 0.1070 x0_111 + 0.0022 x0x0x0_1 + 0.0714 x0x0x0_111
